# explore tweet frequency, price reaction

In [ ]:
##Loading in the data
import pandas as pd

# Load Musk tweets
tweets = pd.read_csv("../data/raw/elon_musk_tweets.csv", parse_dates=["date"])

# Load TSLA price data
prices = pd.read_csv("../data/raw/tsla_ohlcv_daily.csv", skiprows=2, index_col=0, parse_dates=True)
prices.index.name = "Date"
prices = prices.reset_index()

# Quick look
print(tweets.head())
print(prices.head())



                    id  user_name user_location           user_description  \
0  1544379368478212100  Elon Musk           NaN  Mars & Cars, Chips & Dips   
1  1544377493263720450  Elon Musk           NaN  Mars & Cars, Chips & Dips   
2  1544377130590552064  Elon Musk           NaN  Mars & Cars, Chips & Dips   
3  1544375575724400645  Elon Musk           NaN  Mars & Cars, Chips & Dips   
4  1544375148605853699  Elon Musk           NaN  Mars & Cars, Chips & Dips   

                user_created  user_followers  user_friends  user_favourites  \
0  2009-06-02 20:12:29+00:00       101240855           115            13503   
1  2009-06-02 20:12:29+00:00       101240806           115            13503   
2  2009-06-02 20:12:29+00:00       101240806           115            13503   
3  2009-06-02 20:12:29+00:00       101240806           115            13503   
4  2009-06-02 20:12:29+00:00       101240806           115            13503   

   user_verified                      date  \
0         